In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import math
from matplotlib import pyplot

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
url = 'https://drive.google.com/file/d/1mAzX7WF1CIzZVH2AZVZTKM4tNwu-svci/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
train_df = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1Pgz4phglQTq32rwqXNXq2LurWLEFRIHh/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
test_df = pd.read_csv(path)

Let us create a new dataframe with both the train and test data, to make data cleaning easier.

In [ ]:
full_df=pd.concat([train_df, test_df]).reset_index(drop=True)
full_df=full_df.drop('SalePrice',axis=1)

Let's remove the features in which the proportion of null values is greater than 0.25. There are 5 such features.

In [ ]:
b=full_df.isnull().sum().sort_values(ascending=False)/len(full_df)
b=b[full_df.isnull().sum().sort_values(ascending=False)/len(full_df)>0.25]
full_df=full_df.drop(b.index,axis=1)
b

PoolQC         0.996574
MiscFeature    0.964029
Alley          0.932169
Fence          0.804385
FireplaceQu    0.486468
dtype: float64

Now let's fill the rest of the null values with the mode of that feature.

In [ ]:
for a in full_df.columns:
  full_df[a].fillna(full_df[a].mode()[0],inplace=True)

This is something I had to add after running the One Hot Encoding code. For some reason, the last thirteen rows of full_df were being dropped at the end, I was unable to figure out why, so I added thriteen dummy rows.

In [ ]:
for i in range(0,13):
  full_df=full_df.append(full_df.iloc[2918,:],ignore_index=True)

We save a column from full_df now, to make sure that the rows are not messed up later on.

In [ ]:
check=full_df['MSSubClass']

Let's separate the numerical and categorical features and from that separate the ordinal and non-ordinal ones.

In [ ]:
num_features = [f for f in full_df.columns if full_df.dtypes[f] != 'object']
cat_features = [f for f in full_df.columns if full_df.dtypes[f] == 'object']

o_features=['LotShape','Utilities','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','KitchenQual','Functional','GarageFinish','GarageQual','GarageCond','PavedDrive']
no_features = [i for i in cat_features if i not in o_features]

Now, for the non-ordinal features, we do label encoding for the ones with more than 6 features and one hot encoding for the rest. And then we drop the original features and add the encoded features to full_df.

In [ ]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
for f in no_features:
  integer_encoded = label_encoder.fit_transform(full_df[f])
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  if len(full_df[f].unique())>6:
    full_df[f+'_encoded']=integer_encoded
    full_df=full_df.drop(f,axis=1)
  else:
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    lol=pd.DataFrame(onehot_encoded,columns=[f+'_'+s for s in sorted(full_df[f].unique())])
    lol['Id'] = np.arange(lol.shape[0])
    full_df=pd.merge(full_df,lol,on='Id').drop([f,f+'_'+sorted(full_df[f].unique())[-1]],axis=1)

Now we again categorize the ordinal features into easy ordinal features (the ones that have similar general entries) and difficult ordinal features (the ones that have feature specific entries).

In [ ]:
eo_features=[f for f in o_features if(set(sorted(full_df[f].unique())).issubset(set(['Ex','Gd','TA','Fa','Po','NA'])))] 
do_features = [i for i in o_features if i not in eo_features]

Now we encode these manually. Our dataframe full_df, now has only numerical values.

In [ ]:
lol={'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
for f in eo_features:
  full_df[f+'_encoded']=full_df[f].map(lol)
  full_df=full_df.drop(f,axis=1)
  
ordinal_cat_mapping = {
    'BsmtFinType1': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'BsmtFinType2': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'GarageFinish': {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3},
    'PavedDrive': {'N': 0, 'P': 1, 'Y': 2},
    'LotShape': {'IR3': 0,'IR2': 1,'IR1': 2,'Reg': 3},
    'Utilities': {'ELO': 0,'NoSeWa': 1,'NoSewr': 2,'AllPub': 3},
    'LandSlope': {'Sev': 0,'Mod': 1,'Gtl': 2},
    'BsmtExposure': {'NA': 0,'No': 1,'Mn': 2,'Av': 3,'Gd': 4},
    'Functional': {'Sal': 0,'Sev': 1,'Maj2': 2,'Maj1': 3,'Mod': 4,'Min2': 5,'Min1': 6,'Typ': 1}}
    
for ordinal_feature, feature_mapping in ordinal_cat_mapping.items():
    full_df[ordinal_feature + '_encoded'] = full_df[ordinal_feature].map(feature_mapping)
    full_df=full_df.drop(ordinal_feature,axis=1)

In [ ]:
full_df.head(5)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,Foundation_Slab,Foundation_Stone,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,CentralAir_N,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,SaleType_encoded,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,ExterQual_encoded,ExterCond_encoded,BsmtQual_encoded,BsmtCond_encoded,HeatingQC_encoded,KitchenQual_encoded,GarageQual_encoded,GarageCond_encoded,BsmtFinType1_encoded,BsmtFinType2_encoded,GarageFinish_encoded,PavedDrive_encoded,LotShape_encoded,Utilities_encoded,LandSlope_encoded,BsmtExposure_encoded,Functional_encoded
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,1.0,0.0,2,1,3,1,8,0,2003.0,2.0,548.0,0,61,0,0,0,0,0,2,2008,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8,0.0,0.0,0.0,0.0,1.0,4,3,4,3,5,4,3,3,6,1,2,2,3,3,2,1,1
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,0.0,1.0,2,0,3,1,6,1,1976.0,2.0,460.0,298,0,0,0,0,0,0,5,2007,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8,0.0,0.0,0.0,0.0,1.0,3,3,4,3,5,3,3,3,5,1,2,2,3,3,2,4,1
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,434.0,920.0,920,866,0,1786,1.0,0.0,2,1,3,1,6,1,2001.0,2.0,608.0,0,42,0,0,0,0,0,9,2008,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,1.0,0.0,0.0,0.0,0.0,4,3,4,3,5,4,3,3,6,1,2,2,2,3,2,2,1
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,540.0,756.0,961,756,0,1717,1.0,0.0,1,0,3,1,7,1,1998.0,3.0,642.0,0,35,272,0,0,0,0,2,2006,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8,0.0,0.0,0.0,0.0,1.0,3,3,3,4,4,4,3,3,5,1,1,2,2,3,2,1,1
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,1053,0,2198,1.0,0.0,2,1,4,1,9,1,2000.0,3.0,836.0,192,84,0,0,0,0,0,12,2008,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8,0.0,0.0,0.0,0.0,1.0,4,3,4,3,5,4,3,3,6,1,2,2,2,3,2,3,1


Now let us see which are the features that are highly correlated with each other.

In [ ]:
cor=full_df.corr()
corr=cor.unstack().sort_values(ascending=False).drop_duplicates()
[[corr.index[i],corr[i]] for i in range(len(corr)) if corr.index[i][0]!=corr.index[i][1] and corr[i]>0.7]

[[('GarageArea', 'GarageCars'), 0.8881399126047644],
 [('Exterior2nd_encoded', 'Exterior1st_encoded'), 0.8565791840174694],
 [('BsmtFinSF2', 'BsmtFinType2_encoded'), 0.8114131465178478],
 [('GrLivArea', 'TotRmsAbvGrd'), 0.8089883772049516],
 [('TotalBsmtSF', '1stFlrSF'), 0.8012713337541714],
 [('YearBuilt', 'BsmtQual_encoded'), 0.7393522425414609],
 [('OverallQual', 'ExterQual_encoded'), 0.7299640550741138],
 [('KitchenQual_encoded', 'ExterQual_encoded'), 0.7172753091603752],
 [('YearBuilt', 'GarageYrBlt'), 0.71604280007723]]

We remove a few of these manually.

In [ ]:
a=['GarageCars','Exterior2nd_encoded','BsmtFinSF2','TotalBsmtSF','GarageYrBlt',]
full_df=full_df.drop(a,axis=1)

Finally, we drop the dummy rows we added earlier. Somehow, they didn't get removed even though they were the last 13 rows, but when I didn't add them, the last 13 rows were deleted. I really couldn't figure out why this was happening.

In [ ]:
full_df=full_df.drop_duplicates(keep='first').reset_index(drop=True)

After dropping them, we make sure that no other rows in full_df have been compromised.

In [ ]:
(full_df['MSSubClass']-check).sum()

0.0

Now, let us try feature scaling.

In [ ]:
columns_=full_df.columns
id=full_df['Id']
full_df=MinMaxScaler().fit_transform(full_df)
full_df=pd.DataFrame(full_df, columns =columns_)
full_df['Id']=id

In [ ]:
full_df.head(5)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,Street_Grvl,LandContour_Bnk,LandContour_HLS,...,Foundation_Slab,Foundation_Stone,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,CentralAir_N,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,SaleType_encoded,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,ExterQual_encoded,ExterCond_encoded,BsmtQual_encoded,BsmtCond_encoded,HeatingQC_encoded,KitchenQual_encoded,GarageQual_encoded,GarageCond_encoded,BsmtFinType1_encoded,BsmtFinType2_encoded,GarageFinish_encoded,PavedDrive_encoded,LotShape_encoded,Utilities_encoded,LandSlope_encoded,BsmtExposure_encoded,Functional_encoded
0,1,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.12250,0.125089,0.064212,0.109641,0.413559,0.0,0.259231,0.333333,0.0,0.50,0.5,0.375,0.333333,0.461538,0.00,0.368280,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.666667,0.5,0.666667,0.666667,1.00,0.666667,0.5,0.5,1.0,0.0,0.5,1.0,1.000000,1.0,1.0,0.000000,0.0
1,2,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.00000,0.173281,0.121575,0.194917,0.000000,0.0,0.174830,0.000000,0.5,0.50,0.0,0.375,0.333333,0.307692,0.25,0.309140,0.209270,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.333333,0.5,0.666667,0.666667,1.00,0.333333,0.5,0.5,0.8,0.0,0.5,1.0,1.000000,1.0,1.0,1.000000,0.0
2,3,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.10125,0.086109,0.185788,0.123083,0.419370,0.0,0.273549,0.333333,0.0,0.50,0.5,0.375,0.333333,0.307692,0.25,0.408602,0.000000,0.056604,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.666667,0.5,0.666667,0.666667,1.00,0.666667,0.5,0.5,1.0,0.0,0.5,1.0,0.666667,1.0,1.0,0.333333,0.0
3,4,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.00000,0.038271,0.231164,0.131695,0.366102,0.0,0.260550,0.333333,0.0,0.25,0.0,0.375,0.333333,0.384615,0.25,0.431452,0.000000,0.047170,0.268775,0.0,0.0,0.0,0.0,0.090909,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.333333,0.5,0.333333,1.000000,0.75,0.666667,0.5,0.5,0.8,0.0,0.0,1.0,0.666667,1.0,1.0,0.000000,0.0
4,5,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.21875,0.116052,0.209760,0.170342,0.509927,0.0,0.351168,0.333333,0.0,0.50,0.5,0.500,0.333333,0.538462,0.25,0.561828,0.134831,0.113208,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.666667,0.5,0.666667,0.666667,1.00,0.666667,0.5,0.5,1.0,0.0,0.5,1.0,0.666667,1.0,1.0,0.666667,0.0


Now, we split full_df back into the train and test dataframes.

In [ ]:
try1=np.split(full_df,[1460],axis=0)
train=try1[0]
test=try1[1]

Now, we remove outliers by dropping the rows in which the absolute value of the zscore is larger than 7 in any of the columns.

In [ ]:
len(train)

1460

In [ ]:
train=train[(np.abs(stats.zscore(train)) < 7).all(axis=1)]

In [ ]:
len(train)

1194

Now, we add the saleprice column back to train dataframe.

In [ ]:
train['SalePrice']=[train_df['SalePrice'][i] for i in train.index]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Now, we remove the features whose absolute correlation with the saleprice is lesser than 0.1. We drop these 18 columns from train and test dataframes.

In [ ]:
corrr=train.corr()
corrrr=corrr.unstack().sort_values(ascending=False).drop_duplicates()
c=corrrr['SalePrice'][corrrr['SalePrice'].abs()<0.1].abs().sort_values()

train=train.drop(c.index,axis=1)
test=test.drop(c.index,axis=1)
c

GarageType_BuiltIn      0.000741
RoofStyle_Gable         0.006607
LandContour_HLS         0.007027
BldgType_Duplex         0.009541
SaleCondition_Normal    0.016423
LotConfig_CulDSac       0.016798
YrSold                  0.018331
LandContour_Bnk         0.020613
ExterCond_encoded       0.023455
LandContour_Low         0.025888
3SsnPorch               0.044229
GarageType_Attchd       0.044238
MiscVal                 0.047828
SaleType_encoded        0.056621
Electrical_FuseA        0.059587
Condition2_encoded      0.060282
BsmtFinType2_encoded    0.066317
ScreenPorch             0.085030
dtype: float64

Now we split the train dataframe into training and crossval dataframes and training dataframe further into X and y.

In [ ]:
training, crossval = train_test_split(train.drop('Id',axis=1), test_size=0.2)

X=training.drop('SalePrice',axis=1)
y=training['SalePrice']

Now let's try fitting a few models. We try OLS and ridge regression.

In [ ]:
model = sm.OLS(y,X)
results = model.fit()
cvres_ols=results.predict(crossval.drop('SalePrice',axis=1))
np.sqrt(mean_squared_error(crossval['SalePrice'].astype(float),cvres_ols,squared=False))

152.40473554672593

We choose the alpha which gives us the lowest cross validation error.

In [ ]:
cv=[]
for alf in [0,0.025,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]:
  ridgereg = Ridge(alpha=alf, normalize=True)
  ridgereg.fit(X, y)
  dis=np.sqrt(mean_squared_error(crossval['SalePrice'],ridgereg.predict(crossval.drop('SalePrice',axis=1)),squared=False))
  cv.append(dis)
  if dis==np.min(cv):
    best_alpha=alf

In [ ]:
ridgereg = Ridge(alpha=best_alpha, normalize=True)
ridgereg.fit(X, y)
cvres_ridreg = ridgereg.predict(crossval.drop('SalePrice',axis=1))
np.sqrt(mean_squared_error(crossval['SalePrice'].astype(float),cvres_ridreg,squared=False))

151.33123669730503

Now, let us try feature selection on the basis of the p value of each feature we got from OLS earlier. We drop the features that have p value greater than 0.1. There are 45 such features.

In [ ]:
p_value_drop=[f for f in results.pvalues.index if results.pvalues[f]>0.1 or math.isnan(results.pvalues[f])]

training_p=training.drop(p_value_drop,axis=1)
crossval_p=crossval.drop(p_value_drop,axis=1)
test_p=test.drop(p_value_drop,axis=1)

X_p=training_p.drop('SalePrice',axis=1)
y_p=training_p['SalePrice']

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


In [ ]:
model_p = sm.OLS(y_p,X_p)
results_p = model_p.fit()
cvres_ols_p=results_p.predict(crossval_p.drop('SalePrice',axis=1))
np.sqrt(mean_squared_error(crossval_p['SalePrice'].astype(float),cvres_ols_p,squared=False))

151.97398582863548

We choose the alpha which gives us the lowest cross validation error.

In [ ]:
cv=[]
for alf in [0,0.025,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]:
  ridgereg_p = Ridge(alpha=alf, normalize=True)
  ridgereg_p.fit(X_p, y_p)
  dis=np.sqrt(mean_squared_error(crossval_p['SalePrice'],ridgereg_p.predict(crossval_p.drop('SalePrice',axis=1)),squared=False))
  cv.append(dis)
  if dis==np.min(cv):
    best_alpha=alf

In [ ]:
best_alpha

0.3

In [ ]:
ridgereg_p = Ridge(alpha=best_alpha, normalize=True)
ridgereg_p.fit(X_p, y_p)
cvres_ridreg_p = ridgereg_p.predict(crossval_p.drop('SalePrice',axis=1))
np.sqrt(mean_squared_error(crossval_p['SalePrice'].astype(float),cvres_ridreg_p,squared=False))

150.38919686427403

Since ridgereg_p, regularized linear regression with feature selection based on p values has the smallest cross validation error (by a small margin) out of these four models, we select it as the final model.

Now, using the model that we have selected, let us regress on the entire training dataset (without splitting for cross validation set).

In [ ]:
train_p=pd.concat([training_p,crossval_p])

In [ ]:
train_p

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtUnfSF,1stFlrSF,2ndFlrSF,GrLivArea,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,GarageArea,WoodDeckSF,MSZoning_FV,LotConfig_FR3,Neighborhood_encoded,RoofStyle_Mansard,Exterior1st_encoded,Foundation_Stone,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,CentralAir_N,SaleCondition_Abnorml,ExterQual_encoded,BsmtQual_encoded,KitchenQual_encoded,GarageQual_encoded,Utilities_encoded,BsmtExposure_encoded,Functional_encoded,SalePrice
576,0.176471,0.106164,0.023333,0.666667,0.750,0.405797,0.00000,0.000000,0.368579,0.114052,0.290557,0.215335,0.0,0.375,0.333333,0.307692,0.145161,0.000000,0.0,0.0,0.750000,0.0,0.928571,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.333333,0.333333,0.333333,0.50,1.0,0.000000,0.0,145000
1378,0.823529,0.000000,0.003052,0.555556,0.500,0.731884,0.25500,0.054748,0.074486,0.031296,0.244068,0.123022,0.5,0.250,0.333333,0.230769,0.177419,0.050562,0.0,0.0,0.083333,0.0,0.428571,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.50,1.0,0.000000,0.0,83000
483,0.588235,0.037671,0.014957,0.555556,0.500,0.913043,0.07250,0.158930,0.136558,0.185255,0.000000,0.166164,0.0,0.250,0.333333,0.230769,0.270161,0.000000,0.0,0.0,0.458333,0.0,0.857143,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,1.000000,0.333333,0.50,1.0,0.000000,0.0,164000
1293,0.235294,0.195205,0.041319,0.666667,0.500,0.753623,0.00000,0.034373,0.273116,0.104600,0.402906,0.250565,0.5,0.500,0.333333,0.461538,0.354839,0.000000,0.0,0.0,0.583333,0.0,0.428571,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.666667,0.333333,0.50,1.0,0.000000,0.0,162900
403,0.235294,0.246575,0.050434,0.777778,0.500,0.913043,0.40625,0.000000,0.488442,0.174543,0.531719,0.363414,0.5,0.500,0.333333,0.615385,0.282258,0.101124,0.0,0.0,0.625000,0.0,0.857143,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.666667,0.666667,0.666667,0.50,1.0,0.000000,0.0,258000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,0.000000,0.202055,0.038795,0.444444,0.625,0.608696,0.00000,0.089298,0.233733,0.150389,0.000000,0.134891,0.0,0.250,0.333333,0.230769,0.227151,0.000000,0.0,0.0,0.500000,0.0,0.571429,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.50,1.0,0.000000,0.0,128900
1273,0.352941,0.352740,0.047732,0.555556,0.750,0.630435,0.05250,0.127392,0.128425,0.214871,0.000000,0.192728,0.0,0.250,0.333333,0.230769,0.209677,0.000000,0.0,0.0,0.291667,0.0,0.642857,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.333333,1.000000,0.50,1.0,0.666667,0.0,177000
1432,0.058824,0.133562,0.044404,0.333333,0.625,0.398551,0.00000,0.000000,0.280822,0.133165,0.000000,0.119442,0.0,0.500,0.333333,0.230769,0.145161,0.000000,0.0,0.0,0.708333,0.0,0.928571,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.25,1.0,0.000000,0.0,64500
132,0.000000,0.184932,0.028456,0.444444,0.625,0.630435,0.00000,0.071758,0.281678,0.208570,0.000000,0.187076,0.0,0.375,0.333333,0.384615,0.419355,0.000000,0.0,0.0,0.500000,0.0,0.571429,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.666667,0.50,1.0,0.000000,0.0,150750


In [ ]:
ridgereg_p_final = Ridge(alpha=0.3, normalize=True)
ridgereg_p_final.fit(train_p.drop('SalePrice',axis=1), train_p['SalePrice'])

Ridge(alpha=0.3, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [ ]:
pred=ridgereg_p_final.predict(test_p.drop('Id',axis=1))

In [ ]:
pred=pd.DataFrame(pred, columns=['SalePrice'])

In [ ]:
pred=pred.reset_index()

In [ ]:
pred=pred.rename(columns={"index": "Id"})

In [ ]:
pred['Id']=pred['Id']+1461

In [ ]:
pred.to_csv('Submission.csv')

In [ ]:
pred

,Id,SalePrice
0,1461,130318.405389
1,1462,187606.276569
2,1463,174390.138698
3,1464,185515.543745
4,1465,189413.558363
...,...,...
1454,2915,75866.505011
1455,2916,85270.333280
1456,2917,185358.234996
1457,2918,126597.134999


Summarizing what we have done in this notebook:


*   Removed columns with a large proportion of null values.
*   Filled the rest of the null values with the column mode.
*   Used one hot encoding and label encoding to categorical variables into numerical ones.
*   Removed features that have high correlation with each other.
*   Did feature scaling using Min Max Scaler.
*   Removed features whose absolute correlation with the target variable was very small.
*   Used p values from OLS for feature selection.
* Also tried log transformations and feature selection using feature scores from SelectKBest, but they did not seem to help here.
*   Used OLS and Ridge regression models to predict.
*   Calculated cross validation error to find the best regularization parameter and to choose the final model.





